In [ ]:
import os
import random
import shutil
import numpy as np
import pandas as pd
from numpy import copy
import matplotlib.pyplot as plt
from skimage.io import imread_collection,imsave
from sklearn.model_selection import train_test_split
from glob import glob
from scipy.stats import randint as sp_randint
import time
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage.measure import label,regionprops
import copy
from scipy.stats import skew, kurtosis
from skimage.io import imread
from IPython.display import clear_output
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skimage.feature import graycomatrix, graycoprops, hog
from skimage.filters import threshold_otsu, threshold_niblack
from sklearn.cluster import KMeans

from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score, accuracy_score, f1_score
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# Preparando a base
As seguintes celulas são responsáveis por preparar a base de dados para o treinamento do modelo.
Inicialmente, é feita a limpeza dos diretórios de treinamento e teste, para que não haja arquivos de treinamento e teste anteriores. Em seguida, é feita a leitura dos arquivos e a preparação de uma sub-base já com o pre-processamento feito.

In [ ]:
!rm -rf subDataSets dataset.zip Rice_Image_Dataset

In [ ]:
import requests
import zipfile as zf

# dataset url
url = "https://www.muratkoklu.com/datasets/vtdhnd09.php"
print('Downloading dataset.zip')
# download the dataset
r = requests.get(url, allow_redirects=True, stream=True)

with open('dataset.zip', 'wb') as f:
    total_length = int(r.headers['content-length'])
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)
            f.flush()
            print(f'\rDownloaded {f.tell()}/{total_length}', end='')

print('\nDownloaded dataset.zip')
zf.ZipFile('dataset.zip').extractall(members=None, pwd=None)

Downloaded 229550800/229550800
Downloaded dataset.zip


In [ ]:
dataSetPath = "Rice_Image_Dataset"
dataSetClass = [
    'Arborio',
    'Basmati',
    'Ipsala',
    'Jasmine',
    'Karacadag'
]

subDataSetPath = "subDataSets"
subDataSets = [
    "withoutProcessing",
    "negative",
]


In [ ]:
def negative(pixel, l):
    return l - 1 - pixel

In [ ]:
def processImage(img, funcao, *args):
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            img[row][col] = funcao(img[row][col], *args)

    return img

In [ ]:
os.makedirs(subDataSetPath, exist_ok=True)

def createSubDataSet(path, subDataSetName, dataSetClass):
    os.makedirs(os.path.join(path, subDataSetName), exist_ok=True)
    for c in dataSetClass:
        os.makedirs(os.path.join(path, subDataSetName, c), exist_ok=True)

In [ ]:
def copyAndProcessImage(path:str, outPath:str, funcao, *args):
    img = imread(path)
    img = rgb2gray(img)
    img = processImage(img, funcao, *args)
    plt.imsave(outPath, img, cmap="gray")

In [ ]:
def randomImagesPathByClass(path, dataSetClass, n):
    randomImagesPath = {}
    for c in dataSetClass:
        randomImagesPath[c] = []
        for img in random.sample(os.listdir(os.path.join(path, c)), n):
            randomImagesPath[c].append(os.path.join(path, c, img))
    
    return randomImagesPath

In [ ]:
!rm -rf subDataSets/withoutProcessing subDataSets/negative

In [ ]:
imagesQuantity = 200

imagePathByClass = randomImagesPathByClass(dataSetPath, dataSetClass, imagesQuantity)

# gerando subDataSet sem processamento
# createSubDataSet(subDataSetPath, subDataSets[0], dataSetClass)
# for i in imagePathByClass:
#     for img in imagePathByClass[i]:
#         shutil.copy(img, os.path.join(subDataSetPath, subDataSets[0], i))

In [ ]:
createSubDataSet(subDataSetPath, subDataSets[1], dataSetClass)

In [ ]:
count, total = 0, len(imagePathByClass) * imagesQuantity

for i in imagePathByClass.keys():
    for img in imagePathByClass[i]:
        count += 1
        print(f"\rProcessing {count} of {total}", end="")

        l = rgb2gray(imread(img)).max()

        copyAndProcessImage(
            img,
            os.path.join(subDataSetPath, subDataSets[1], i, os.path.basename(img)),
            negative,
            l
        )

Processing 1000 of 1000

# Preparando a segmentação
Será usado Outsu e Niblack para segmentação da imagem. A segmentação é feita em duas etapas, uma para a imagem original e outra para a imagem com o pre-processamento feito.

In [ ]:
def applyOtsu(img):
    img = img.copy()
    if len(img.shape) == 3:
        img = rgb2gray(img)
    threshold = threshold_otsu(img)
    img = img > threshold
    return img

In [ ]:
def getLabel(imgPath):
    if 'Arborio' in imgPath:
        return 0
    elif 'Basmati' in imgPath:
        return 1
    elif 'Ipsala' in imgPath:
        return 2
    elif 'Jasmine' in imgPath:
        return 3
    elif 'Karacadag' in imgPath:
        return 4
    
def getClassName(label):
    if label == 0:
        return 'Arborio'
    elif label == 1:
        return 'Basmati'
    elif label == 2:
        return 'Ipsala'
    elif label == 3:
        return 'Jasmine'
    elif label == 4:
        return 'Karacadag'

## Carregando imagens

In [ ]:
pathNegative = 'subDataSets/negative'

negativeClassPaths =  (
    glob(os.path.join(pathNegative, 'Arborio', '*.jpg'))
    + glob(os.path.join(pathNegative, 'Basmati', '*.jpg'))
    + glob(os.path.join(pathNegative, 'Ipsala', '*.jpg'))
    + glob(os.path.join(pathNegative, 'Jasmine', '*.jpg'))
    + glob(os.path.join(pathNegative, 'Karacadag', '*.jpg'))
    )

len(negativeClassPaths)

1000

## Extraindo features

In [ ]:
def extract_features_with_glcm(paths):
    featuresGLCM = np.zeros((len(paths),18)) #6 features x 3 color channels
    labelsGLCM = np.zeros(len(paths))

    start = time.time()

    d = 2

    processedImgs = 0

    for id_im, imgPath in enumerate(paths):
        imagem = imread(imgPath)
        mask = applyOtsu(imagem)

        imagem[:,:,0] = imagem[:,:,0] * mask
        imagem[:,:,1] = imagem[:,:,1] * mask
        imagem[:,:,2] = imagem[:,:,2] * mask

        labelsGLCM[id_im] = getLabel(imgPath)

        for id_ch in range(3):
            matrix0 = graycomatrix(imagem[:,:,id_ch], [d], [0],normed=True)
            matrix1 = graycomatrix(imagem[:,:,id_ch], [d], [np.pi/4],normed=True)
            matrix2 = graycomatrix(imagem[:,:,id_ch], [d], [np.pi/2],normed=True)
            matrix3 = graycomatrix(imagem[:,:,id_ch], [d], [3*np.pi/4],normed=True)
            matrix = (matrix0+matrix1+matrix2+matrix3)/4

            matrix[0, 0, 0, 0] = 0

            props = np.zeros((6))
            props[0] = graycoprops(matrix,'contrast')
            props[1] = graycoprops(matrix,'dissimilarity')
            props[2] = graycoprops(matrix,'homogeneity')
            props[3] = graycoprops(matrix,'energy')
            props[4] = graycoprops(matrix,'correlation')
            props[5] = graycoprops(matrix,'ASM')
            featuresGLCM[id_im, id_ch*6 : (id_ch+1)*6] = props

        processedImgs += 1
        print(f"Label: {getLabel(imgPath)} - index: {getClassName(labelsGLCM[id_im])} - Processing {processedImgs} of {imagesQuantity * 5}", end="\r")

    end = time.time()
    print("")
    print(end - start)

    return featuresGLCM, labelsGLCM

In [ ]:
def extract_features_with_hog(paths):
    featuresHOG = np.zeros((len(paths), 36)) # 36 features
    labelsHOG = np.zeros(len(paths))

    orientations = 8
    pixels_per_cell = (16, 16)
    cells_per_block = (1, 1)

    featuresHOG = []

    for id_im, imagePath in enumerate(paths):
        image = imread(imagePath)
        labelsHOG[id_im] = getLabel(image)
        fd = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block, feature_vector=True, multichannel=True)
        
        featuresHOG.append(fd)

    featuresHOG = np.array(featuresHOG)

    return featuresHOG, labelsHOG

In [ ]:
featureHOG, labelHOG = extract_features_with_hog(negativeClassPaths)

In [ ]:
featuresGLCM, labelsGLCM = extract_features_with_glcm(negativeClassPaths)

Label: 4 - index: Karacadag - Processing 1000 of 1000
28.02224612236023


In [ ]:
k = 5

kf_hog = KFold(n_splits=k, shuffle=True)
x_pca_hog = PCA(n_components=1).fit_transform(featuresGLCM)

rfMetricsHOG = pd.DataFrame(data=[], columns=['Accuracy', 'Kappa', 'f1'])
mlpMetricsHOG = pd.DataFrame(data=[], columns=['Accuracy', 'Kappa', 'f1'])

for train_index, test_index in kf_hog.split(x_pca_hog):
    
    X_train, X_test = x_pca_hog[train_index], x_pca_hog[test_index]
    y_train, y_test = labelsGLCM[train_index], labelsGLCM[test_index]
    
    rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    rf.fit(X_train, y_train)
    
    acc_rf = accuracy_score(y_test, rf.predict(X_test))
    kp_rf = cohen_kappa_score(y_test, rf.predict(X_test))
    f1_rf = f1_score(y_test, rf.predict(X_test), average='weighted')

    rfMetricsHOG = rfMetricsHOG.append({'Accuracy': acc_rf, 'Kappa': kp_rf, 'f1': f1_rf}, ignore_index=True)

    mlp = MLPClassifier()
    mlp.fit(X_train, y_train)

    acc_mlp = accuracy_score(y_test, mlp.predict(X_test))
    kp_mlp = cohen_kappa_score(y_test, mlp.predict(X_test))
    f1_mlp = f1_score(y_test, mlp.predict(X_test), average='weighted')

    mlpMetricsHOG = mlpMetricsHOG.append({'Accuracy': acc_mlp, 'Kappa': kp_mlp, 'f1': f1_mlp}, ignore_index=True)

In [ ]:
k = 5

kf_glcm = KFold(n_splits=k, shuffle=True)
x_pca_glcm = PCA(n_components=1).fit_transform(featuresGLCM)

rfMetricsGLCM = pd.DataFrame(data=[], columns=['Accuracy', 'Kappa', 'f1'])
mlpMetricsGLCM = pd.DataFrame(data=[], columns=['Accuracy', 'Kappa', 'f1'])

for train_index, test_index in kf_glcm.split(x_pca_glcm):
    
    X_train, X_test = x_pca_glcm[train_index], x_pca_glcm[test_index]
    y_train, y_test = labelsGLCM[train_index], labelsGLCM[test_index]
    
    rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    rf.fit(X_train, y_train)
    
    acc_rf = accuracy_score(y_test, rf.predict(X_test))
    kp_rf = cohen_kappa_score(y_test, rf.predict(X_test))
    f1_rf = f1_score(y_test, rf.predict(X_test), average='weighted')

    rfMetricsGLCM = rfMetricsGLCM.append({'Accuracy': acc_rf, 'Kappa': kp_rf, 'f1': f1_rf}, ignore_index=True)

    mlp = MLPClassifier()
    mlp.fit(X_train, y_train)

    acc_mlp = accuracy_score(y_test, mlp.predict(X_test))
    kp_mlp = cohen_kappa_score(y_test, mlp.predict(X_test))
    f1_mlp = f1_score(y_test, mlp.predict(X_test), average='weighted')

    mlpMetricsGLCM = mlpMetricsGLCM.append({'Accuracy': acc_mlp, 'Kappa': kp_mlp, 'f1': f1_mlp}, ignore_index=True)

In [ ]:
rfMetricsGLCM

,Accuracy,Kappa,f1
0,0.680,0.602139,0.610667
1,0.705,0.628335,0.645028
2,0.775,0.717496,0.752109
3,0.695,0.618082,0.629920
4,0.695,0.620399,0.612370


In [ ]:
mlpMetricsGLCM

,Accuracy,Kappa,f1
0,0.705,0.630742,0.623125
1,0.625,0.528909,0.588864
2,0.810,0.761329,0.802756
3,0.690,0.612015,0.688825
4,0.685,0.609955,0.600222


In [ ]:
mlpMetricsGLCM.mean()

Accuracy    0.703000
Kappa       0.628590
f1          0.660758
dtype: float64

In [ ]:
mlpMetricsGLCM.std()

Accuracy    0.067138
Kappa       0.083938
f1          0.088319
dtype: float64

In [ ]:
rfMetricsGLCM.mean()

Accuracy    0.710000
Kappa       0.637290
f1          0.650019
dtype: float64

In [ ]:
rfMetricsGLCM.std()

Accuracy    0.037417
Kappa       0.045834
f1          0.058773
dtype: float64

In [ ]:
mlpMetricsHOG

,Accuracy,Kappa,f1
0,0.74,0.672565,0.673289
1,0.74,0.670459,0.687375
2,0.69,0.608968,0.675536
3,0.75,0.687314,0.749247
4,0.68,0.597800,0.643892


In [ ]:
rfMetricsHOG

,Accuracy,Kappa,f1
0,0.715,0.640662,0.646035
1,0.755,0.690295,0.687649
2,0.675,0.596674,0.595431
3,0.705,0.631595,0.632255
4,0.700,0.624248,0.631209


In [ ]:
mlpMetricsHOG.mean()

Accuracy    0.720000
Kappa       0.647421
f1          0.685868
dtype: float64

In [ ]:
mlpMetricsHOG.std()

Accuracy    0.032404
Kappa       0.040912
f1          0.038879
dtype: float64

In [ ]:
rfMetricsHOG.mean()

Accuracy    0.710000
Kappa       0.636695
f1          0.638516
dtype: float64

In [ ]:
rfMetricsHOG.std()

Accuracy    0.029155
Kappa       0.034173
f1          0.033240
dtype: float64